In [ ]:
# Install the necessary library to interact with BigQuery
%pip install pandas-gbq
%pip install --upgrade google-genai

In [ ]:
import pandas_gbq
import bigframes.pandas as bf
import os

In [ ]:
service_account = "demo-real-estate@adcz-adoki-poc.iam.gserviceaccount.com"

### Get data from BigQuery
- using local/Workbanch computing

In [ ]:
# Use local computing power
sql = "SELECT * FROM demo_real_estate.property"
df_property = bf.read_gbq(sql)

# display(df_property.head())

In [ ]:
# Use local computing power
sql = "SELECT * FROM demo_real_estate.property_type"
df_type = bf.read_gbq(sql)

# display(df_type.head())

### Filter data

In [ ]:
# Merge
merged_df = df_property.merge(df_type, left_on='property_type_id', right_on='property_type_key', how='left')

# Filtr
filter_df = merged_df[merged_df['type_name'].str.lower() == 'byt']

# Final filter
final_df = filter_df[['property_id', 'description']].head(10)

display(final_df)

### Define Gemini prompt-description fc

In [ ]:
from google import genai
from google.genai import types
import base64

def call_ai(description):
  client = genai.Client(
      vertexai=True,
      project="adcz-adoki-poc",
      location="global",
  )

  prompt = description if description is not None else ""

  msg1_text1 = types.Part.from_text(text=f"""{prompt}""")
  si_text1 = """You will be provided with description of a property that is listed on a real-estate website. Only print the size of the property in square meters. If not provided print 'None'"""

  model = "gemini-2.5-flash-lite"
  contents = [
    types.Content(
      role="user",
      parts=[
        msg1_text1
      ]
    ),
  ]

  generate_content_config = types.GenerateContentConfig(
    temperature = 0,
    top_p = 0.95,
    max_output_tokens = 1000,
    safety_settings = [types.SafetySetting(
      category="HARM_CATEGORY_HATE_SPEECH",
      threshold="OFF"
    ),types.SafetySetting(
      category="HARM_CATEGORY_DANGEROUS_CONTENT",
      threshold="OFF"
    ),types.SafetySetting(
      category="HARM_CATEGORY_SEXUALLY_EXPLICIT",
      threshold="OFF"
    ),types.SafetySetting(
      category="HARM_CATEGORY_HARASSMENT",
      threshold="OFF"
    )],
    system_instruction=[types.Part.from_text(text=si_text1)],
    thinking_config=types.ThinkingConfig(
      thinking_budget=0,
    ),
  )

  collected = []
  for chunk in client.models.generate_content_stream(
    model = model,
    contents = contents,
    config = generate_content_config,
    ):
    if chunk.text:
      print(chunk.text, end="")
      collected.append(chunk.text)

  result = ''.join(collected).strip()
  return result if result else None

### Call the fc and get results

In [ ]:
final_df_local = final_df.to_pandas()  # materiálizace do paměti (pozor na velikost!)
final_df_local['property_size'] = final_df_local['description'].apply(call_ai)

display(final_df_local)